In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D, InputLayer
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle 

import multiprocessing

import os
import cv2
import time
import gc

from PIL import Image

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus: 
    try: # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus: 
            tf.config.experimental.set_memory_growth(gpu, True)
            
        logical_gpus = tf.config.experimental.list_logical_devices('GPU') 
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs") 
    
    except RuntimeError as e: # Memory growth must be set before GPUs have been initialized 
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
# keras reproducible 
import random

class config:
    seed = 13
    
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything(config.seed)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1, 
    inter_op_parallelism_threads=1
)
sess = tf.compat.v1.Session(
    graph=tf.compat.v1.get_default_graph(), 
    config=session_conf
)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
def model_2(input_shape=(64, 64, 1)):

    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())  # 3d array from CNN layer to 1d array 
    model.add(Dense(128))
    model.add(Activation('relu'))
    
    model.add(Dense(128))
    model.add(Activation('relu'))
    
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.SGD(),
                  metrics=['accuracy'])
    return model

In [ ]:
model = model_2()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0

In [ ]:
batch_size = 16
epochs = 200


x_train = np.load('../data/2c/x_train.npy')
y_train = np.load('../data/2c/y_train.npy')
x_valid = np.load('../data/2c/x_valid.npy')
y_valid = np.load('../data/2c/y_valid.npy')

# x_train = x_train.reshape(x_train.shape[0], 64, 64, 1)
# x_valid = x_valid.reshape(x_valid.shape[0], 64, 64, 1)

input_shape = (64, 64, 2)

x_train, y_train = shuffle(x_train, y_train, random_state=13) 

y_train = y_train.astype(np.uint8)
y_valid = y_valid.astype(np.uint8)

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
valid_generator = valid_datagen.flow(x_valid, y_valid, batch_size=batch_size)

# for i in range(2, 3):
model = model_2(input_shape=input_shape)
model.summary()
# model checkpoint

mc = ModelCheckpoint('mc/2c/ori-loss-model.h5', monitor='val_loss', mode='min', save_best_only=True)

# train -- fit
history = model.fit(
train_generator,
epochs=epochs,
validation_data=valid_generator,
callbacks=[mc],
workers=8
)

/home/miruware/anaconda3/envs/pcs/lib/python3.8/site-packages/keras_preprocessing/image/numpy_array_iterator.py:129: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (2188, 64, 64, 2) (2 channels).
  warnings.warn('NumpyArrayIterator is set to use the '
/home/miruware/anaconda3/envs/pcs/lib/python3.8/site-packages/keras_preprocessing/image/numpy_array_iterator.py:129: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (564, 64, 64, 2) (2 channels).
  warnings.warn('NumpyArrayIterator is set to use the '


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 32)        608       
_________________________________________________________________
activation_12 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 64)       

## DataAugmentation 적용해보기 
## ImageDataGenerator사용하거나, 직접 만들어보기